## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
!pip install -r requirements.txt -q

In [ ]:
from xgboost import XGBClassifier
import pandas as pd

from functions.utils import (
    load_image,
    show_image
)
from functions.donut import (
    load_cheque_parser,
    parse_text,
    evaluate_cheque_fraud,
)
from functions.llm_chain import(
    get_llm_chain,
    generate_response,
    format_response,
)
from features.cheque_validation import get_cheque_ids

import config

import warnings
warnings.filterwarnings('ignore')

## <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

## <span style="color:#ff5f27;"> 🪝 Download the Fraud Detection Model from Model Registry </span>

In [ ]:
mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="cheque_fraud_detection_model",
    version=1,
)

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

In [ ]:
model_fraud_detection = XGBClassifier()

model_fraud_detection.load_model(saved_model_dir + "/model.json")

## <span style='color:#ff5f27'> 🍩 Donut Model Loading </span>


In [ ]:
processor, model_parser = load_cheque_parser(
    config.DONUT_FT_REPO,
)

## <span style='color:#ff5f27'> 🚀 Cheque Text Parsing </span>


In [ ]:
image1 = load_image('1.jpg')

parsed_text1 = parse_text(
    image1, 
    processor, 
    model_parser,
)
print(parsed_text1)

show_image(image1)

In [ ]:
image2 = load_image('2.jpg')

parsed_text2 = parse_text(
    image2, 
    processor, 
    model_parser,
)
print(parsed_text2)

show_image(image2)

In [ ]:
image502 = load_image('502.jpg')

parsed_text502 = parse_text(
    image502, 
    processor, 
    model_parser,
)
print(parsed_text502)

show_image(image502)

In [ ]:
image503 = load_image('503.jpg')

parsed_text503 = parse_text(
    image503, 
    processor, 
    model_parser,
)
print(parsed_text503)

show_image(image503)

## <span style='color:#ff5f27'>👨🏻‍⚖️ Check Evaluation </span>


In [ ]:
evaluate_cheque_fraud(
    parsed_text1, 
    model_fraud_detection,
)

In [ ]:
evaluate_cheque_fraud(
    parsed_text2, 
    model_fraud_detection,
)

In [ ]:
image3 = load_image('3.jpg')

parsed_text3 = parse_text(
    image3, 
    processor, 
    model_parser,
)

evaluate_cheque_fraud(
    parsed_text3, 
    model_fraud_detection,
)

In [ ]:
evaluate_cheque_fraud(
    parsed_text502, 
    model_fraud_detection,
)

In [ ]:
evaluate_cheque_fraud(
    parsed_text503, 
    model_fraud_detection,
)

## <span style='color:#ff5f27'>🔗 LLM Chain Loading </span>


In [ ]:
llm_chain = get_llm_chain()

## <span style='color:#ff5f27'>🚀 Inference </span>


In [ ]:
response = generate_response(
    '2.jpg', 
    processor, 
    model_parser, 
    model_fraud_detection, 
    llm_chain,
    verbose=True,
)
print(response)

In [ ]:
response = generate_response(
    '3.jpg', 
    processor, 
    model_parser, 
    model_fraud_detection, 
    llm_chain,
    verbose=True,
)
print(response)

In [ ]:
response = generate_response(
    '502.jpg', 
    processor, 
    model_parser, 
    model_fraud_detection, 
    llm_chain,
    verbose=True,
)
print(response)

In [ ]:
response = generate_response(
    '503.jpg', 
    processor, 
    model_parser, 
    model_fraud_detection, 
    llm_chain,
    verbose=True,
)
print(response)

## <span style='color:#ff5f27'>🗄️ Batch Inference </span>


In [ ]:
from os import listdir
from os.path import isfile, join

FOLDER_NAME = 'cheques_batch/'

cheque_names = [image for image in listdir(FOLDER_NAME)]
cheque_names

In [ ]:
cheque_batch_validation = [
    generate_response(
        cheque_name, 
        processor, 
        model_parser, 
        model_fraud_detection, 
        llm_chain,verbose=False,
        folder_name=FOLDER_NAME,
    ) 
    for cheque_name 
    in cheque_names
]

responses_formatted = [
    format_response(response) 
    for response 
    in cheque_batch_validation
]

cheque_validation_df = pd.DataFrame(
    responses_formatted, 
    columns=['status', 'description'],
)

cheque_validation_df.head()

## <span style="color:#ff5f27;"> 🪄 Feature Group Creation </span>

In [ ]:
from hsfs.feature import Feature

features = [
    Feature(name="cheque_id", type="bigint"),
    Feature(name="status", type="string"),
    Feature(name="description", type="string", online_type = "TEXT"),
]

In [ ]:
# Get or create the 'cheque_validation' feature group
cheque_validation_fg = fs.get_or_create_feature_group(
    name="cheque_validation_fg",
    version=1,
    description='Cheque Validation and description',
    primary_key=['cheque_id'],
    online_enabled=True,
    features=features,
)

In [ ]:
cheque_validation_df_w_index = get_cheque_ids(
    cheque_validation_fg, 
    cheque_validation_df,
)
cheque_validation_df_w_index.head(3)

In [ ]:
cheque_validation_fg.insert(
    cheque_validation_df_w_index,
)

---